In [1]:
from models import PS_FCN_CBN, PS_FCN 
from datasets import PS_Blobby_BRDF_Dataset 
import torch 
import re

model_cbn = PS_FCN_CBN(c_in=6) 
model = PS_FCN(c_in=6) 
# checkpoint = torch.load('/data2/datasets/ruoguli/idl_project_datas/PS-FCN_B_S_32.pth.tar')
# checkpoint['state_dict']
for key in model.state_dict().keys():
	print(key)
print('------------------------')
for key in model_cbn.state_dict().keys():
	print(key)
# model.load_state_dict(checkpoint) 
	

def loadCheckpoint_to_PSFCN_CBN(path, model, cuda=True):
    """
    Loads weights from a pretrained PS_FCN checkpoint into a PS_FCN_CBN model.

    Handles the mapping between standard Conv layers in PS_FCN's extractor
    and the Conditional_Conv layers (specifically the 'regular_conv' part)
    in PS_FCN_CBN's extractor. Assumes batchNorm=False was used for the
    original model's convolutional layers that are being mapped.
    """
    if cuda:
        checkpoint = torch.load(path)
    else:
        checkpoint = torch.load(path, map_location=lambda storage, loc: storage)

    print(f"Loading pretrained weights for PS_FCN_CBN from: {path}")
    pretrained_state_dict = checkpoint['state_dict']
    model_dict = model.state_dict()

    # 1. Create a new state dict to hold the weights to load
    new_state_dict = {}
    loaded_keys = set()
    skipped_keys_mismatch = set()
    skipped_keys_nonexistent = set()
    original_keys_mapped = set()

    # 2. Iterate through the pretrained state dict
    for k_pretrained, v_pretrained in pretrained_state_dict.items():
        original_keys_mapped.add(k_pretrained) # Track keys from original model

        # Try direct mapping first (covers Regressor and extractor.conv6, extractor.conv7)
        if k_pretrained in model_dict and model_dict[k_pretrained].shape == v_pretrained.shape:
            new_state_dict[k_pretrained] = v_pretrained
            loaded_keys.add(k_pretrained)
            # print(f"Directly loaded: {k_pretrained}")
            continue # Go to next key

        # --- Special Mapping for extractor.conv1 to conv5 ---
        # Check if it's a weight from the original extractor's conv layers 1-5
        match = re.match(r'extractor\.conv(\d+)\.0\.weight', k_pretrained)
        if match:
            layer_num = match.group(1)
            # Construct the corresponding key in the new model
            k_new = f'extractor.conv{layer_num}.regular_conv.weight'

            if k_new in model_dict:
                if model_dict[k_new].shape == v_pretrained.shape:
                    new_state_dict[k_new] = v_pretrained
                    loaded_keys.add(k_new)
                    # print(f"Mapped: {k_pretrained} -> {k_new}")
                else:
                    # print(f"Shape mismatch: {k_pretrained} ({v_pretrained.shape}) vs {k_new} ({model_dict[k_new].shape})")
                    skipped_keys_mismatch.add(k_pretrained)
            else:
                # print(f"Key not found in new model: {k_new} (mapped from {k_pretrained})")
                skipped_keys_nonexistent.add(k_pretrained)
            continue # Go to next key

        # Check if it's a bias from the original extractor's conv layers 1-5
        # These biases don't exist in the new model's 'regular_conv' (bias=False)
        match_bias = re.match(r'extractor\.conv(\d+)\.0\.bias', k_pretrained)
        if match_bias and int(match_bias.group(1)) <= 5 :
            # print(f"Skipped (no bias in new layer): {k_pretrained}")
            skipped_keys_nonexistent.add(k_pretrained) # Bias intentionally skipped
            continue # Go to next key

        # If we reach here, the key was not directly matched or specifically mapped/skipped
        # print(f"Skipped (no match/mapping rule): {k_pretrained}")
        if k_pretrained in model_dict:
             skipped_keys_mismatch.add(k_pretrained) # Likely shape mismatch if key exists but wasn't caught above
        else:
             skipped_keys_nonexistent.add(k_pretrained) # Key doesn't exist in target


    # 3. Update the model's state dict with the mapped weights
    model_dict.update(new_state_dict)

    # 4. Load the updated state dict into the model
    model.load_state_dict(model_dict)

    # 5. Report Summary
    print("-" * 50)
    print("State Dict Loading Summary:")
    print(f"  Loaded {len(loaded_keys)} weights.")
    # Optionally print details
    # print("  Loaded Keys:", sorted(list(loaded_keys)))

    print(f"  Skipped {len(skipped_keys_mismatch)} pretrained keys due to shape mismatch.")
    # print("  Skipped Mismatch Keys:", sorted(list(skipped_keys_mismatch)))

    print(f"  Skipped {len(skipped_keys_nonexistent)} pretrained keys (no target layer/param or intentionally ignored like old biases).")
    # print("  Skipped Non-existent/Ignored Keys:", sorted(list(skipped_keys_nonexistent)))

    # Check for keys in the new model that were *not* loaded
    model_keys = set(model.state_dict().keys())
    not_loaded = model_keys - loaded_keys
    print(f"  {len(not_loaded)} keys in the new model were not loaded from checkpoint (initialized instead).")
    # print("  Not Loaded Keys:", sorted(list(not_loaded))) # This will include CBN params, BRDF embedder, etc.
    print("-" * 50)
    



Conv pad = 1
Conv pad = 1
Conv pad = 1
Conv pad = 1
Conv pad = 1
Conv pad = 1
Conv pad = 1
Conv pad = 1
Conv pad = 1
Conv pad = 1
Conv pad = 1
extractor.conv1.0.weight
extractor.conv1.0.bias
extractor.conv2.0.weight
extractor.conv2.0.bias
extractor.conv3.0.weight
extractor.conv3.0.bias
extractor.conv4.0.weight
extractor.conv4.0.bias
extractor.conv5.0.weight
extractor.conv5.0.bias
extractor.conv6.0.weight
extractor.conv7.0.weight
extractor.conv7.0.bias
regressor.deconv1.0.weight
regressor.deconv1.0.bias
regressor.deconv2.0.weight
regressor.deconv2.0.bias
regressor.deconv3.0.weight
regressor.est_normal.0.weight
------------------------
brdf_embed_block.embed_transform.weight
brdf_embed_block.embed_transform.bias
extractor.conv1.regular_conv.weight
extractor.conv1.regular_conv.bias
extractor.conv1.cBatchNorm.betas
extractor.conv1.cBatchNorm.gammas
extractor.conv1.cBatchNorm.running_mean
extractor.conv1.cBatchNorm.running_var
extractor.conv1.cBatchNorm.num_batches_tracked
extractor.conv1.c

In [2]:
import re
import time # For unique log file names

# --- Keep your existing model_utils functions here ---
# getInput, parseData, getInputChanel, get_n_params, loadCheckpoint_to_PSFCN,
# saveCheckpoint, conv, deconv


# --- Add this UPDATED DETAILED logging version ---
def loadCheckpoint_to_PSFCN_CBN_debug(path, model, cuda=True, output_log_file=None):
    """
    Loads weights from a pretrained PS_FCN checkpoint into a PS_FCN_CBN model.
    Logs detailed information about loaded, skipped, and uninitialized weights
    to the specified output_log_file.

    Handles mapping for weights AND biases for extractor.conv1-5.
    Assumes original model used model_utils.conv(batchNorm=False).
    """
    if output_log_file is None:
        timestamp = time.strftime("%Y%m%d-%H%M%S")
        output_log_file = f"loading_log_{timestamp}.txt"

    log_lines = []
    log_lines.append(f"--- Loading Checkpoint: {path} ---")
    log_lines.append(f"--- Target Model: {type(model).__name__} ---")
    log_lines.append("-" * 50)

    if cuda:
        checkpoint = torch.load(path)
    else:
        checkpoint = torch.load(path, map_location=lambda storage, loc: storage)

    log_lines.append(f"Checkpoint loaded successfully from: {path}")
    pretrained_state_dict = checkpoint.get('state_dict', checkpoint) # Handle cases where checkpoint might just be the state_dict
    model_dict = model.state_dict()
    new_model_keys = set(model_dict.keys())

    new_state_dict = {}
    loaded_keys_list = []
    skipped_mismatch_list = []
    skipped_nonexistent_list = []
    # removed skipped_bias_list as biases are now loaded
    original_keys_processed = set()

    log_lines.append("\n--- Processing Pretrained Parameters ---")
    for k_pretrained, v_pretrained in pretrained_state_dict.items():
        original_keys_processed.add(k_pretrained)
        processed = False

        # --- Special Mapping for extractor.conv1 to conv5 Weights ---
        match_weight = re.match(r'extractor\.conv(\d+)\.0\.weight', k_pretrained)
        if match_weight and int(match_weight.group(1)) <= 5:
            layer_num = match_weight.group(1)
            k_new = f'extractor.conv{layer_num}.regular_conv.weight'
            if k_new in model_dict:
                if model_dict[k_new].shape == v_pretrained.shape:
                    new_state_dict[k_new] = v_pretrained
                    loaded_keys_list.append(f"{k_pretrained} -> {k_new} (Shape: {tuple(v_pretrained.shape)})")
                    processed = True
                else:
                    skipped_mismatch_list.append(f"{k_pretrained} (Shape: {tuple(v_pretrained.shape)}) vs {k_new} (Shape: {tuple(model_dict[k_new].shape)})")
                    processed = True
            else:
                 skipped_nonexistent_list.append(f"{k_pretrained} (Target key {k_new} not found in new model)")
                 processed = True
            # Continue to next key in outer loop IF processed, otherwise might be handled by bias rule or direct match
            if processed: continue

        # --- Special Mapping for extractor.conv1 to conv5 Biases ---
        match_bias = re.match(r'extractor\.conv(\d+)\.0\.bias', k_pretrained)
        if match_bias and int(match_bias.group(1)) <= 5:
            layer_num = match_bias.group(1)
            k_new = f'extractor.conv{layer_num}.regular_conv.bias' # Target is now the bias of regular_conv
            if k_new in model_dict:
                if model_dict[k_new].shape == v_pretrained.shape:
                    new_state_dict[k_new] = v_pretrained
                    loaded_keys_list.append(f"{k_pretrained} -> {k_new} (Shape: {tuple(v_pretrained.shape)})")
                    processed = True
                else:
                    skipped_mismatch_list.append(f"{k_pretrained} (Shape: {tuple(v_pretrained.shape)}) vs {k_new} (Shape: {tuple(model_dict[k_new].shape)})")
                    processed = True
            else:
                 skipped_nonexistent_list.append(f"{k_pretrained} (Target key {k_new} not found in new model)")
                 processed = True
            # Continue to next key in outer loop IF processed
            if processed: continue

        # --- Try Direct Mapping for other keys (Regressor, extractor.conv6/7 etc.) ---
        # This rule will now handle keys like extractor.conv7.0.bias directly too.
        if not processed:
            if k_pretrained in model_dict:
                if model_dict[k_pretrained].shape == v_pretrained.shape:
                    new_state_dict[k_pretrained] = v_pretrained
                    loaded_keys_list.append(f"{k_pretrained} -> {k_pretrained} (Direct Match, Shape: {tuple(v_pretrained.shape)})")
                    processed = True
                else:
                    skipped_mismatch_list.append(f"{k_pretrained} (Shape: {tuple(v_pretrained.shape)}) vs {k_pretrained} (Shape: {tuple(model_dict[k_pretrained].shape)})")
                    processed = True
                # Continue to next key in outer loop IF processed
                if processed: continue

        # --- If not processed by any rule above, it's skipped as non-existent in target ---
        if not processed:
             skipped_nonexistent_list.append(f"{k_pretrained} (No matching key or rule in new model)")


    log_lines.append("\n--- Loading into Model ---")
    model_dict.update(new_state_dict)
    model.load_state_dict(model_dict, strict=True) # strict=True ensures all model_dict keys are consumed
    log_lines.append("model.load_state_dict(model_dict) executed.")


    # --- Report Summary ---
    log_lines.append("\n" + "=" * 50)
    log_lines.append("      State Dict Loading Summary")
    log_lines.append("=" * 50)

    # Loaded Keys
    log_lines.append(f"\n--- ({len(loaded_keys_list)}) Keys Loaded Successfully ---")
    loaded_keys_list.sort()
    log_lines.extend(loaded_keys_list)

    # Skipped Keys (Shape Mismatch)
    log_lines.append(f"\n--- ({len(skipped_mismatch_list)}) Pretrained Keys Skipped (Shape Mismatch) ---")
    skipped_mismatch_list.sort()
    log_lines.extend(skipped_mismatch_list)

    # Skipped Keys (Non-Existent Target)
    log_lines.append(f"\n--- ({len(skipped_nonexistent_list)}) Pretrained Keys Skipped (No Target Layer/Param) ---")
    skipped_nonexistent_list.sort()
    log_lines.extend(skipped_nonexistent_list)

    # Keys in New Model NOT Loaded
    final_loaded_keys = set(new_state_dict.keys())
    not_loaded_new_keys = new_model_keys - final_loaded_keys
    not_loaded_list = sorted(list(not_loaded_new_keys))
    log_lines.append(f"\n--- ({len(not_loaded_list)}) Keys in New Model NOT Loaded from Checkpoint (Initialized Instead) ---")
    log_lines.extend(not_loaded_list)
    log_lines.append("=" * 50)


    # --- Write log to file ---
    try:
        with open(output_log_file, 'w') as f:
            for line in log_lines:
                f.write(line + '\n')
        print(f"Detailed loading log saved to: {output_log_file}")
    except IOError as e:
        print(f"Error writing log file {output_log_file}: {e}")
        print("\n--- Log Output (Console) ---")
        for line in log_lines:
            print(line)

# --- Also update the standard (non-debug) loader for consistency ---
def loadCheckpoint_to_PSFCN_CBN(path, model, cuda=True):
    """
    Loads weights from a pretrained PS_FCN checkpoint into a PS_FCN_CBN model.
    Handles mapping for weights AND biases for extractor.conv1-5.
    """
    if cuda:
        checkpoint = torch.load(path)
    else:
        checkpoint = torch.load(path, map_location=lambda storage, loc: storage)

    print(f"Loading pretrained weights for PS_FCN_CBN from: {path} (using standard loader)")
    pretrained_state_dict = checkpoint.get('state_dict', checkpoint)
    model_dict = model.state_dict()
    new_state_dict = {}
    loaded_count = 0
    skipped_count = 0

    for k_pretrained, v_pretrained in pretrained_state_dict.items():
        mapped = False
        # Special Mapping for extractor.conv1 to conv5 Weights
        match_weight = re.match(r'extractor\.conv(\d+)\.0\.weight', k_pretrained)
        if match_weight and int(match_weight.group(1)) <= 5:
            layer_num = match_weight.group(1)
            k_new = f'extractor.conv{layer_num}.regular_conv.weight'
            if k_new in model_dict and model_dict[k_new].shape == v_pretrained.shape:
                new_state_dict[k_new] = v_pretrained
                mapped = True
            if mapped: continue

        # Special Mapping for extractor.conv1 to conv5 Biases
        match_bias = re.match(r'extractor\.conv(\d+)\.0\.bias', k_pretrained)
        if match_bias and int(match_bias.group(1)) <= 5:
            layer_num = match_bias.group(1)
            k_new = f'extractor.conv{layer_num}.regular_conv.bias'
            if k_new in model_dict and model_dict[k_new].shape == v_pretrained.shape:
                new_state_dict[k_new] = v_pretrained
                mapped = True
            if mapped: continue

        # Direct mapping for others
        if not mapped:
            if k_pretrained in model_dict and model_dict[k_pretrained].shape == v_pretrained.shape:
                new_state_dict[k_pretrained] = v_pretrained
                mapped = True

        if mapped:
            loaded_count += 1
        else:
            skipped_count += 1


    model_dict.update(new_state_dict)
    model.load_state_dict(model_dict, strict=True) # Use strict=True

    print(f"Standard loading complete. Loaded {loaded_count} parameters, skipped {skipped_count} from pretrained dict.")
    print(f"{len(model.state_dict()) - len(new_state_dict)} parameters in the new model were initialized.")
loadCheckpoint_to_PSFCN_CBN_debug('/data2/datasets/ruoguli/idl_project_datas/PS-FCN_B_S_32.pth.tar', model_cbn, cuda=True)


/tmp/ipykernel_1029293/591810211.py:29: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(path)


RuntimeError: Attempting to deserialize object on a CUDA device but torch.cuda.is_available() is False. If you are running on a CPU-only machine, please use torch.load with map_location=torch.device('cpu') to map your storages to the CPU.